In [1]:
# нагрузка по работе не позволяет выполнить до конца работу, однако скоро решение будет закомиченно.  
# 

In [ ]:
#Домашнее задание по теме «Рекомендации на основе содержания»
#Преподаватель: Александр Кондрашкин
#
#    1. Использовать dataset MovieLens
#    2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
#
#    3. TF-IDF на тегах и жанрах
#    4. Средние оценки (+ median, variance, etc.) пользователя и фильма
#
#    5. Оценить RMSE на тестовой выборке



In [123]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# Фичи будут складываться из tf-idf на жанрах и тегах,  

In [21]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [29]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [81]:
def change_string(l):
    return '|'.join(l).replace(' ', '').replace('-', '')

In [82]:
# для построение векторов на базе tfidf совместим таблици в одну

tags_gr = pd.DataFrame(tags.groupby('movieId').apply(lambda x: change_string(list(x['tag']))), columns=['tags'])

In [83]:
movie_tag_gnr = pd.merge(movies, tags_gr, on='movieId', how='left' ).fillna('')
movie_tag_gnr['tags_gnr'] = movie_tag_gnr.apply(lambda x: x['genres']+ ('|'+str(x['tags'])  if str(x['tags']) != '' else ''), axis=1)
movie_tag_gnr.head()

,movieId,title,genres,tags,tags_gnr
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun,Adventure|Animation|Children|Comedy|Fantasy|pi...
1,2,Jumanji (1995),Adventure|Children|Fantasy,fantasy|magicboardgame|RobinWilliams|game,Adventure|Children|Fantasy|fantasy|magicboardg...
2,3,Grumpier Old Men (1995),Comedy|Romance,moldy|old,Comedy|Romance|moldy|old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,,Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy,pregnancy|remake,Comedy|pregnancy|remake


In [72]:
# Для генерацити фич испорльзуем следующие моменты
# из тегов
# -- средняя длина тега
# -- количество тегов (с повторами)
# -- для учета параметров времени выставления тегов
# -- среднее время между тегами
# -- время прошедшее с первого выставления тага
# -- время прошедшее с последнего выставления тага
#
# -- количестиво пользователей поставивших тег 
# -- среднее количество выставления тегов на пользователя
#
# из жанров
# -- количество жанров
# 
# -- год производства 


result = pd.DataFrame(movies['movieId'])
result.shape

(9742, 1)

In [57]:
# -- средняя длина тега
tags['len_tag'] = tags.apply(lambda x: len(x['tag'].split(' ')), axis=1)
result['mean_len_tag'] = tags[['len_tag','movieId']].groupby('movieId').mean()

,movieId,mean_len_tag,count_tag
0,1,0.00,0.0
1,2,1.00,3.0
2,3,1.75,4.0
3,4,1.00,2.0
4,5,0.00,0.0


In [89]:
# -- количество тегов (с повторами)
result['count_tag'] = tags[['tag','movieId']].groupby('movieId').count()
result.fillna(0, inplace=True)
result.head()

,movieId,count_tag
0,1,0.0
1,2,3.0
2,3,4.0
3,4,2.0
4,5,0.0


In [104]:
import time

curent_time = time.time()
print(curent_time)
# -- среднее время прошедшее между выставлением тегов
result['avg_time'] = tags.sort_values(['movieId','timestamp']).groupby('movieId').apply(lambda x: (x['timestamp'].iloc[1:]-x['timestamp'].iloc[:-1]).mean() if len(x['timestamp']) > 1 else 0 )
result.fillna(0, inplace=True)


# -- время первого тега прошедшее между выставлением тегов
result['first_time'] = curent_time - tags[['movieId','timestamp']].groupby('movieId').min()
result['last_time'] = curent_time - tags[['movieId','timestamp']].groupby('movieId').max()
result.fillna(0, inplace=True)
result.head(10)

1555448407.264699


,movieId,count_tag,avg_time,min_time,first_time,last_time
0,1,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
1,2,3.0,0.0,4.182415e+08,4.182416e+08,3.016239e+07
2,3,4.0,0.0,4.180728e+08,4.180729e+08,2.660448e+07
3,4,2.0,0.0,4.120235e+08,4.120235e+08,4.120235e+08
4,5,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
5,6,2.0,0.0,4.180745e+08,4.180745e+08,4.180745e+08
6,7,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
7,8,1.0,0.0,4.180727e+08,4.180728e+08,4.180728e+08
8,9,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00
9,10,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00


In [115]:
result['count_user'] = tags[['movieId','userId']].groupby('movieId').count()
result.fillna(0, inplace=True)
result.head()

,movieId,count_tag,avg_time,min_time,first_time,last_time,count_user,count_genres
0,1,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,5
1,2,3.0,0.0,4.182415e+08,4.182416e+08,3.016239e+07,3.0,3
2,3,4.0,0.0,4.180728e+08,4.180729e+08,2.660448e+07,4.0,2
3,4,2.0,0.0,4.120235e+08,4.120235e+08,4.120235e+08,2.0,3
4,5,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1


In [114]:
result['count_genres'] = movies['genres'].apply(lambda x: len(x.split('|')))
result.head()

,movieId,count_tag,avg_time,min_time,first_time,last_time,count_user,count_genres
0,1,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,5
1,2,3.0,0.0,4.182415e+08,4.182416e+08,3.016239e+07,3.0,3
2,3,4.0,0.0,4.180728e+08,4.180729e+08,2.660448e+07,4.0,2
3,4,2.0,0.0,4.120235e+08,4.120235e+08,4.120235e+08,2.0,3
4,5,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,1


In [108]:
movie_tag_gnr.shape

(9742, 5)

In [145]:
 y_ratings = pd.merge(movies, ratings, on='movieId', how='left' )[['movieId','rating']].groupby('movieId').agg({'rating':['mean','std'] }).reset_index()

In [146]:
y_ratings.head()

movieId    rating          
               mean       std
0       1  3.920930  0.834859
1       2  3.431818  0.881713
2       3  3.259615  1.054823
3       4  2.357143  0.852168
4       5  3.071429  0.907148

In [147]:
y_ratings.shape

(9742, 3)

In [148]:
y_ratings['rating']['mean'].shape

(9742,)

In [84]:
tfidf_transformer = TfidfVectorizer()
X_train_tfidf = tfidf_transformer.fit_transform(movie_tag_gnr['tags_gnr'])

In [85]:
X_train_tfidf.shape

(9742, 1480)

In [119]:
np.array(result).shape

(9742, 8)

In [120]:
from scipy.sparse import hstack
X_train_tfidf = hstack((X_train_tfidf,np.array(result)))

In [137]:
X_train_tfidf.shape

(9742, 1488)

In [153]:
X,  X_test,y, y_test = train_test_split(X_train_tfidf,y_ratings['rating'], test_size=0.3 )

In [155]:
y_train.head()

,mean,std
9130,4.000000,NaN
1350,4.500000,0.500000
5265,3.491803,0.928672
9114,3.500000,NaN
8539,2.500000,NaN


In [156]:
# выберем модели

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ARDRegression

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
param_model_GS = {'Ridge':{'alpha':[0.1,0.5,1]}, 
              'Lasso':{'alpha':[0.1,0.5,1]}, 
              'KNeighborsRegressor':{'n_neighbors':[3,5,7,9]},
              'LinearSVR':{'C':[0.1,0.5,1], 'loss':['epsilon_insensitive', 'squared_epsilon_insensitive']},
              'DecisionTreeRegressor':{'max_depth':[3,5,7,9]},                                    
              'LinearRegression':{},
              'RandomForestRegressor': {'max_depth':[3,5,7,9]},
              'GradientBoostingRegressor':{'loss' : ['ls', 'lad', 'huber', 'quantile']}   
             }

Rg  = GridSearchCV(Ridge(), param_model_GS['Ridge'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)
Ls  = GridSearchCV(Lasso(), param_model_GS['Lasso'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)
KNR = GridSearchCV(KNeighborsRegressor(), param_model_GS['KNeighborsRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)
SVR = GridSearchCV(LinearSVR(), param_model_GS['LinearSVR'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)
DTR = GridSearchCV(DecisionTreeRegressor(), param_model_GS['DecisionTreeRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)
LR  = LinearRegression().fit(X,y).predict(X_out)
RFR = GridSearchCV(RandomForestRegressor(), param_model_GS['RandomForestRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)
GBR = GridSearchCV(GradientBoostingRegressor(), param_model_GS['GradientBoostingRegressor'], cv=5, n_jobs=-1).fit(X,y).best_estimator_.predict(X_test)


In [ ]:
print('ridge   :', mean_absolute_error(Rg,y_test))
print('lasso   :', mean_absolute_error(Ls,y_test))
print('KNR     :', mean_absolute_error(KNR,y_test))
print('SVR     :', mean_absolute_error(SVR,y_test))
print('DTR     :', mean_absolute_error(DTR,y_test))
print('LR      :', mean_absolute_error(LR,y_test))
print('RFR     :', mean_absolute_error(RFR,y_test))
print('GBR     :', mean_absolute_error(GBR,y_test))


In [ ]:
print('ridge   :', r2_score(Rg,y_test))
print('lasso   :', r2_score(Ls,y_test))
print('KNR     :', r2_score(KNR,y_test))
print('SVR     :', r2_score(SVR,y_test))
print('DTR     :', r2_score(DTR,y_test))
print('LR      :', r2_score(LR,y_test))
print('RFR     :', r2_score(RFR,y_test))
print('GBR     :', r2_score(GBR,y_test))
